# Circularity vs Stellar Age

In [92]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [93]:
from auriga.images import figure_setup
from auriga.snapshot import Snapshot
from auriga.settings import Settings

In [94]:
figure_setup()
settings = Settings()

#### Config

In [95]:
N_BINS = 200
MIN_PART_COUNT = 1E1
MAX_PART_COUNT = 1E3

#### `read_data`

In [96]:
def read_data(simulation: str) -> np.ndarray:
    """
    This method returns a data frame with the circularity (`Circularity`) and
    the stellar age in Gyr (`StellarAge_Gyr`) for each star. Note that
    `StellarAge_Gyr` is the stellar age and not the formation time.

    Parameters
    ----------
    simulation : str
        The simulation to load.

    Returns
    -------
    data : pd.DataFrame
        The data frame.
    """
    s = Snapshot(simulation=simulation, loadonlytype=[0, 1, 2, 3, 4, 5])
    s.add_circularity()
    s.add_stellar_age()

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    data = pd.DataFrame({
        "Circularity": s.circularity[is_real_star & is_main_obj],
        "StellarAge_Gyr": s.stellar_age[is_real_star & is_main_obj]
    })

    return data

#### Figures

In [97]:
def plot_map_for_sample() -> None:
    fig = plt.figure(figsize=(7, 8))
    gs = fig.add_gridspec(nrows=6, ncols=4, hspace=0.0, wspace=0.0)
    axs = gs.subplots(sharex=True, sharey=True)

    for ax in axs.flatten():
        ax.tick_params(which='both', direction="in")
        if ax == axs[-1, -1]: ax.axis("off")
        ax.set_xlim(0, 14)
        ax.set_xticks([2, 4, 6, 8, 10, 12])
        ax.set_ylim(-1.4, 1.4)
        ax.set_yticks([-1, -0.5, 0, 0.5, 1.0])
        ax.set_axisbelow(True)
        if ax.get_subplotspec().is_last_row() or ax == axs[-2, -1]:
            ax.set_xlabel("Stellar Age [Gyr]")
            ax.tick_params(labelbottom=True)
        if ax.get_subplotspec().is_first_col():
            ax.set_ylabel(r'$\epsilon = j_z \, j_\mathrm{circ}^{-1}$')

    for i in range(len(settings.groups["Included"])):
        ax = axs.flatten()[i]
        galaxy = settings.groups["Included"][i]
        label = f"Au{galaxy}"
        simulation = f"au{galaxy}_or_l4_s127"
        df = read_data(simulation=simulation)
        _, _, _, qm = ax.hist2d(
            df["StellarAge_Gyr"], df["Circularity"],
            cmap='nipy_spectral',
            bins=N_BINS,
            range=[ax.get_xlim(), ax.get_ylim()],
            norm=mcolors.LogNorm(vmin=MIN_PART_COUNT, vmax=MAX_PART_COUNT),
            rasterized=True)

        ax.text(x=0.05, y=0.95, s=r"$\texttt{" + label + "}$", color="black",
                size=6.0, transform=ax.transAxes, ha='left', va='top')

        if galaxy == 2:
            cbar = plt.colorbar(qm, ax=axs.flatten()[-1], shrink=0.8,
                                location="bottom", pad=3)
            cbar.set_ticks([1E1, 1E2, 1E3])
            cbar.set_label(f"$N_\star$")

        fig.savefig("../images/circularity_vs_stellar_age/included.pdf")

    plt.close(fig)

In [98]:
plot_map_for_sample()